<a href="https://colab.research.google.com/github/aditya248/Netflix-Exploratory-Data-Analysis/blob/main/BA775_TeamProject_B2T8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#BA775 - Team Project - Team 8
##*Project Proposal*

###**Summary Research for Netflix on it’s library**

Our mission is to help Netflix summarize the details of their movie and TV show library to help them see trends on how their library has changed. We will also summarize Netflix materials based on popular genres and actors to compare with highest earning  and popular entertainment types so Netflix can maximize their user amount. 

**Source:**
https://www.kaggle.com/shivamb/netflix-sh


In [ ]:
%%bigquery --ba775-team8-b2


In [ ]:
%%bigquery --ba775-team8-b2


In [ ]:
%%bigquery --ba775-team8-b2


In [ ]:
%%bigquery --ba775-team8-b2
